In [ ]:
import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
import pandas as pd

# 检测输入数据

In [ ]:
BSE = yf.download('MME=F', '2022-01-01', '2022-08-22')['Close']
ITC = yf.download('^BSESN', '2022-01-01', '2022-08-22')['Close']
print(type(BSE))
BSE

In [ ]:
# to calcualte the return fo teh ITC and BSE sensex
ITC = ITC.pct_change()
BSE = BSE.pct_change()
ITC= pd.DataFrame(ITC)
BSE = pd.DataFrame(BSE)
BSE =BSE.dropna(axis=0)
ITC =ITC.dropna(axis=0)

# To have equal counts in ITC & BSE
df = ITC['Close'],BSE['Close']
df1 = pd.DataFrame(df)
df2 = df1.T
df2.columns = ['ITC','BSE']
df3 = df2.dropna()
df3

# 模拟输入数据

In [ ]:
###todo：数据要求
###todo：交易日期为 index，每一列的 column name 为股票代码，每一列是 close【收盘价】的percentage changed【百分比变化，相对于前一天】
stockData=pd.read_csv("stockData/TushareRawData.csv")
stockData=stockData[['ts_code','trade_date','pct_chg']].rename(columns={'trade_date':'date'})
# companyList=[]
afterColumnChanged=pd.DataFrame()
# print("total number of day: {}".format(len(stockData['Date'].values)))
# for stockCode in list(stockData.groupby('ts_code')):
#     theData={'Date':stockCode[1]['Date'].values,stockCode[0]:stockCode[1]['pct_chg'].values}
#     dataByCompany=pd.DataFrame(theData)
#     dataByCompany['Date']=pd.to_datetime(dataByCompany["Date"], format="%Y-%m-%d")
#     dataByCompany=dataByCompany.dropna(axis=1)
#     # dataByCompany=dataByCompany.set_index('Date')
#     # afterColumnChanged=afterColumnChanged.append(dataByCompany)
#     companyList.append(dataByCompany)
#     # afterColumnChanged=pd.concat([afterColumnChanged,dataByCompany])
#     # print("{} is finished".format(stockCode[0]))
#
# afterColumnChanged=pd.concat(companyList[:100])
# # afterColumnChanged=afterColumnChanged.drop_duplicates()
# afterColumnChanged=afterColumnChanged.sort_values(by='Date',ascending=False)
# # afterColumnChanged=afterColumnChanged.dropna(how='all')
stockData['date']=pd.to_datetime(stockData["date"], format="%Y-%m-%d")
stockData=stockData.set_index('date')
# dt = datetime.datetime(2022, 12, 30)
# # for ele in afterColumnChanged[afterColumnChanged['Date']==dt].values:
# #     print(ele)
# afterColumnChanged

for stockCode in stockData.groupby('ts_code'):
    # if len(stockCode[1]['pct_chg'].values) >= 1000:
    afterColumnChanged[stockCode[0]]=stockCode[1]['pct_chg']

# afterColumnChanged=afterColumnChanged.dropna()
afterColumnChanged

In [ ]:
# afterColumnChanged.to_csv("eventStudyData/train/incomeImport.csv",sep=',',index =True)

In [ ]:
afterColumnChanged=pd.read_csv("eventStudyData/train/incomeImport.csv",index_col="date")
print(afterColumnChanged.index.dtype)
afterColumnChanged

# 模拟三因子数据 an aggregate of event

In [ ]:
eventAggregate=pd.DataFrame()
companyTickers=afterColumnChanged.columns
companyTickers

In [ ]:
spy=yf.download('SPY', '2022-01-01', '2022-08-22')['Close']

In [ ]:
fama=pd.read_csv("eventStudyData/train/FamaFrench.csv")
fama

## 输入数据

In [ ]:
es.Single.import_returns('eventStudyData/train/incomeImport.csv')
es.Single.import_FamaFrench('eventStudyData/train/FamaFrench.csv')

In [ ]:
event = es.Single.FamaFrench_3factor(
    security_ticker = '000001.SZ',
    event_date = np.datetime64('2020-01-20'),
    event_window = (-2,+10),
    estimation_size = 300,
    buffer_size = 30
)

In [ ]:
event.plot(AR=True)
plt.show()

# 筛选 5 只医药股（随机）

In [ ]:
rawCompanyData=pd.read_csv("stockData/rawCompanyDataInTushare.csv")
print(rawCompanyData['industry'].unique())
rawMedicCompanyData=rawCompanyData[rawCompanyData['industry']=='医疗保健'].head()['ts_code'].values.tolist()
rawMedicCompanyData

In [ ]:
##todo：时间点：2020/1/20，2020/2/3
eventTime=['2020-1-20','2020-3-20']
# eventTime=['20/1/2020']
def createEventTime(eventTime,security_ticker,market_ticker):
    final=pd.DataFrame()
    for event in eventTime:
        for ticker in security_ticker:
            thisLine=pd.DataFrame({"security_ticker":[ticker],"market_ticker":[market_ticker],"event_date":[event]})
            final=pd.concat([final,thisLine])
    return final

medicEventData=createEventTime(eventTime,rawMedicCompanyData,"SPY")
medicEventData

In [ ]:
##导出
# medicEventData.to_csv("eventStudyData/timeLineData/medicHeadData.csv",index=False)

# 筛选 5 只旅游股（随机）

In [ ]:
rawTravelCompanyData=rawCompanyData[rawCompanyData['industry']=='旅游服务'].head()['ts_code'].values.tolist()
rawTravelCompanyData

In [ ]:
travelEventData=createEventTime(eventTime,rawTravelCompanyData,"SPY")
travelEventData

In [ ]:
# travelEventData.to_csv("eventStudyData/timeLineData/travelHeadData.csv",index=False)

# 合并

In [ ]:
combine=pd.concat([travelEventData,medicEventData])
# combine.to_csv("eventStudyData/timeLineData/combineData.csv",index=False)
combine

# 开始测试 multiple class 接口

In [ ]:
agg = es.Multiple.from_csv(
    path = 'eventStudyData/timeLineData/medicHeadData.csv',
    event_study_model = es.Single.market_model,
    event_window = (-5,+10),
    date_format = "%Y-%m-%d",
    ignore_errors=True
)
agg.plot()
plt.show()